In [125]:
1+1

2

In [126]:
!pip install arxiv

python(6898) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [177]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [178]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [179]:
!pip install wikipedia


python(7064) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [180]:
wiki.name

'wikipedia'

In [181]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())
retriever=vectordb.as_retriever()
retriever

/var/folders/qd/37rs6y8d6sxfjyvthxb_rj3h0000gn/T/ipykernel_3175/2878865163.py:9: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  vectordb=FAISS.from_documents(documents,HuggingFaceEmbeddings())


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x14200f890>, search_kwargs={})

In [182]:
## Created a retrievel tool to search info related to langsmith
from langchain_core.tools import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")


In [183]:
retriever_tool.name

'langsmith_search'

In [184]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [185]:
tools=[wiki,arxiv,retriever_tool]

In [137]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/keerthanamedaveni/Desktop/GenAi-Project/.venv/lib/python3.13/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x13aefa0c0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorsto

In [190]:
from dotenv import load_dotenv

load_dotenv()
import os
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [191]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [192]:
### Agents
from langchain.agents import create_agent
agent=create_agent(llm,tools)

In [193]:
## Agent Executer
from langchain.agents.agent import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=<langgraph.graph.state.CompiledStateGraph object at 0x14204d7b0>, input_keys_arg=[], return_keys_arg=[], stream_runnable=True), tools=[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/keerthanamedaveni/Desktop/GenAi-Project/.venv/lib/python3.13/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)), ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)), Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.

In [196]:
agent_executor = (prompt.partial(agent_scratchpad=[""]) | agent)
res = agent_executor.invoke({"input": "Tell me about LangSmith"})
print(res)


{'messages': [SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}, id='049776d1-0a29-4343-bf35-0a2436930374'), HumanMessage(content='Tell me about LangSmith', additional_kwargs={}, response_metadata={}, id='2abfd57b-e033-48f5-89d7-6e3c3fcc8f4e'), HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='4da283ca-770c-4f5f-8c1e-17bce0bb90a5'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 213, 'total_tokens': 229, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CbZTEELdHsPR5AgUPisLFpdPcjlVi', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': Non

In [209]:
res["messages"][5].type


'ai'

In [210]:
res["messages"][5].content

'LangSmith is a platform designed for developing, debugging, and deploying large language model (LLM) applications. It provides an integrated workflow that combines observability, evaluation, deployment, and platform setup—from local development to production. Key features of LangSmith include:\n\n1. **Data Security and Compliance**: LangSmith adheres to high standards of data security and privacy, being compliant with HIPAA, SOC 2 Type 2, and GDPR.\n\n2. **Integrated Workflow**: It allows developers to trace requests, evaluate outputs, test prompts, and manage deployments all in one place. This integration helps in building more reliable AI systems.\n\n3. **Framework Agnostic**: LangSmith can be used independently or alongside LangChain’s open-source libraries, which offers flexibility in development.\n\n4. **Monitoring and Evaluation**: The platform provides tools for integrated monitoring and evaluation, allowing developers to prototype applications locally and scale them to product

In [212]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})["messages"][5].content

'The paper titled "Heat-bath random walks with Markov bases," authored by Caprice Stanley and Tobias Windisch, was published on May 27, 2016. It examines graphs on lattice points where the edges are derived from a finite set of allowances. If you need more specific details or insights from the paper, please let me know!'